<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import urllib.parse as req

In [2]:
DBNAME = ***
USER = ***
PASSWORD = '***
HOST = ***
PORT = ***

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
query_3_1 = f'''
            select
                count(v.id) as "Количество вакансий"
            from
                public.vacancies as v
'''

In [5]:
vacancies_count = pd.read_sql_query(query_3_1, connection)
vacancies_count

C:\Users\22775\AppData\Local\Temp\ipykernel_18048\3020254417.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vacancies_count = pd.read_sql_query(query_3_1, connection)


,Количество вакансий
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
query_3_2 = f'''
            select
                count(e.id) as "Количество работодателей"
            from
                public.employers as e
'''

In [7]:
employers_count = pd.read_sql_query(query_3_2, connection)
employers_count

C:\Users\22775\AppData\Local\Temp\ipykernel_18048\1997002725.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employers_count = pd.read_sql_query(query_3_2, connection)


,Количество работодателей
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
query_3_3 = f'''
            select
                count(a.id) as "Количество регионов"
            from
                public.areas as a
'''

In [9]:
areas_count = pd.read_sql_query(query_3_3, connection)
areas_count

C:\Users\22775\AppData\Local\Temp\ipykernel_18048\2678816896.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  areas_count = pd.read_sql_query(query_3_3, connection)


,Количество регионов
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
query_3_4 = f'''
            select
                count(i.id) as "Количество сфер деятельности"
            from
                public.industries as i
'''

In [11]:
industires_count = pd.read_sql_query(query_3_4, connection)
industires_count

C:\Users\22775\AppData\Local\Temp\ipykernel_18048\465217737.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  industires_count = pd.read_sql_query(query_3_4, connection)


,Количество сфер деятельности
0,294


***

По предварительному анализу данных мы можем сделать вывод, что у нас имеется срез базы данных с 49197 вакансиями в 1362 регионах от 23501 работодателей, которые распределены по 294 сферам деятельности. Предварительно можно сказать, что количество вакансий больше, чем количество работодателей, а значит  каждый работодатель имеет как минимум больше одной вакансии и скорее всего в нескольких регионах, возможен также вариант одной вакансии в нескольких регионах.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [12]:
query_4_1 = f'''
            select
                a.name as "Регион",
                count(v.id) as "Количество вакансий"
            from
                public.vacancies as v
            join public.areas as a on v.area_id = a.id
            group by a.id
            order by 2 desc
            limit 5
'''

In [13]:
region_chart = pd.read_sql_query(query_4_1, connection)
region_chart

C:\Users\22775\AppData\Local\Temp\ipykernel_18048\3297410438.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  region_chart = pd.read_sql_query(query_4_1, connection)


,Регион,Количество вакансий
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [14]:
query_4_2 = f'''
            select
                count(v.id) as "Количество вакансий"
            from
                public.vacancies as v
            where
                v.salary_from is not null or v.salary_to is not null
'''

In [15]:
salary_filled = pd.read_sql_query(query_4_2, connection)
salary_filled

C:\Users\22775\AppData\Local\Temp\ipykernel_18048\468891745.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  salary_filled = pd.read_sql_query(query_4_2, connection)


,Количество вакансий
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [16]:
query_4_3 = f'''
            select
                round(avg(v.salary_from)) as "Средняя минимальная зарплата",
                round(avg(v.salary_to)) as "Средняя максимальная зарплата"
            from
                public.vacancies as v
'''

In [17]:
avg_sallary = pd.read_sql_query(query_4_3, connection)
avg_sallary

C:\Users\22775\AppData\Local\Temp\ipykernel_18048\1785775701.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  avg_sallary = pd.read_sql_query(query_4_3, connection)


,Средняя минимальная зарплата,Средняя максимальная зарплата
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [18]:
query_4_4 = f'''
            select
                v.schedule as "График работы",
                v.employment as "Тип занятости",
                count(v.id) as "Количество вакансий"
            from
                public.vacancies as v
            group by 1,2
            order by 3 desc
            ---limit 1
            ---offset 1
'''

In [19]:
schedule_employment = pd.read_sql_query(query_4_4, connection)
schedule_employment

C:\Users\22775\AppData\Local\Temp\ipykernel_18048\767197040.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  schedule_employment = pd.read_sql_query(query_4_4, connection)


,График работы,Тип занятости,Количество вакансий
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [20]:
query_4_5 = f'''
            select
                v.experience as "Опыт работыи",
                count(v.id) as "Количество вакансий"
            from public.vacancies as v
            group by 1
            order by 2 asc
'''

In [21]:
experience = pd.read_sql_query(query_4_5, connection)
experience

C:\Users\22775\AppData\Local\Temp\ipykernel_18048\644794075.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  experience = pd.read_sql_query(query_4_5, connection)


,Опыт работыи,Количество вакансий
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

По результатам детального анализа вакансий становится ясно, что основная доля вакансий сосредоточена в крупных городах и столицах. Также в базе присутствуют вакансии с локацией не в России, что говорит о присутствии российских компаний с офисами не в России либо зарубежных компаний с представительствами в РФ. Более чем у половины вакансий не указана заработная плата как её нижний порог так и верхний, что говорит о том, что заработная плата будет определяться по результатам собеседования и будет зависеть от навыков кандидата, либо заработная плата будет за фактически выполненный объем работ. У вакансий с указанными пределами заработной платы разброс между средними показателями коридора заработной платы не сильно большой, что говорит о том, что мы имеем пласт вакансий примерно одного уровня без руководящих или других топ должностей. Видим картину распределения графика и типа занятости, естественным выглядит, что большую часть занимает тип Полная занятость - Полный рабочий день, а вот второе место Удаленная работа - Полная занятость, говорит о большом количестве вакансий с возможностью контроля результата. (IT, Фриланс, Дизайн...). Так же мы видим, что работодатели в основном ищут кандидатов с минимальным опытом работы от 1 до 3 лет, что говорит о поиске людей для дальнейшего обучения и замещения ими должностей, для которых отсутствует необходимость большого опыта работы (Junior специалисты - в нашем случае). Второе место в этом срезе от 3 до 6 лет выглядит как возможность набора людей следующей ступени рабочего опыта. (Middle специалисты - в нашем случае).

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [22]:
query_5_1 = f'''
            select
                e.name as "Название работодателя",
                count(v.id) as "Количество вакансий"
            from
                public.vacancies as v
            join public.employers as e on v.employer_id = e.id
            group by e.id
            order by 2 desc
            limit 5
'''

In [23]:
employers_chart = pd.read_sql_query(query_5_1, connection)
employers_chart

C:\Users\22775\AppData\Local\Temp\ipykernel_18048\1185223683.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employers_chart = pd.read_sql_query(query_5_1, connection)


,Название работодателя,Количество вакансий
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [24]:
query_5_2 = f'''
            select
                a.name as "Название региона",
                count(distinct e.id) as "Количество работодателей",
                count(distinct v.id) as "Количество вакансий"
            from
                public.areas as a
            full outer join public.vacancies as v on a.id = v.area_id
            join public.employers as e on a.id = e.area
            group by a.id
            order by 3 asc, 2 desc
            ---limit 1
'''

In [25]:
region_grouping = pd.read_sql_query(query_5_2, connection)
region_grouping

C:\Users\22775\AppData\Local\Temp\ipykernel_18048\3959748256.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  region_grouping = pd.read_sql_query(query_5_2, connection)


,Название региона,Количество работодателей,Количество вакансий
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0
...,...,...,...
649,Алматы,721,1892
650,Новосибирск,573,2006
651,Минск,1115,2112
652,Санкт-Петербург,2217,2851


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [26]:
query_5_3 = f'''
            select
                e.name,
                count(distinct v.area_id)
            from
                public.vacancies as v
            join public.employers as e on v.employer_id = e.id
            group by e.name
            order by 2 desc
'''

In [27]:
employer_regions = pd.read_sql_query(query_5_3, connection)
employer_regions

C:\Users\22775\AppData\Local\Temp\ipykernel_18048\743286884.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employer_regions = pd.read_sql_query(query_5_3, connection)


,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [30]:
query_5_4 = f'''
            select
                count(e.id)
            from
                public.employers as e
            full outer join public.employers_industries as ei on e.id = ei.employer_id
            where ei.industry_id is null
'''

In [31]:
null_industies_employers = pd.read_sql_query(query_5_4, connection)
null_industies_employers

C:\Users\22775\AppData\Local\Temp\ipykernel_51304\3149523439.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  null_industies_employers = pd.read_sql_query(query_5_4, connection)


,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [32]:
query_5_5 = f'''
            select
                e.name,
                count(ei.industry_id)
            from
                public.employers as e
            full outer join public.employers_industries as ei on e.id = ei.employer_id
            group by e.id
            having count(ei.industry_id) = 4
            order by 1 asc
            limit 1
            offset 2
'''

In [33]:
industries_count = pd.read_sql_query(query_5_5, connection)
industries_count

C:\Users\22775\AppData\Local\Temp\ipykernel_51304\3160677322.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  industries_count = pd.read_sql_query(query_5_5, connection)


,name,count
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [34]:
query_5_6 = f'''
            select
                i.name,
                count(e.employer_id)
            from public.employers_industries as e
            join public.industries as i on e.industry_id = i.id
            where i.name = 'Разработка программного обеспечения'
            group by i.name
'''

In [35]:
software_development = pd.read_sql_query(query_5_6, connection)
software_development

C:\Users\22775\AppData\Local\Temp\ipykernel_51304\3433297533.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  software_development = pd.read_sql_query(query_5_6, connection)


,name,count
0,Разработка программного обеспечения,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [36]:
path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров')
url = path_common.scheme + '://' + req.quote(path_common.netloc) +  req.quote(path_common.path) + '#'+ req.quote(path_common.fragment)
million_list = list(pd.read_html(url)[1]['Город'])
for i,n in enumerate(million_list):
    million_list[i] = (million_list[i]).strip('[a]')
million_list = tuple(million_list)

In [37]:
query_5_7 = f'''
            with x as (
            select
                a.name as "Название региона",
                count(v.id) as vacancies_count
            from
                public.vacancies as v
            join public.areas as a on v.area_id=a.id
            join public.employers as e on v.employer_id = e.id
            where e.name='Яндекс' and a.name in {million_list}
            group by a.name
            order by 2 desc
            )

            select *
            from x

            union all

            select
                'total', sum(x.vacancies_count)
            from x
'''


In [38]:
yandex_vac_count = pd.read_sql_query(query_5_7, connection)
yandex_vac_count

C:\Users\22775\AppData\Local\Temp\ipykernel_51304\3245053768.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  yandex_vac_count = pd.read_sql_query(query_5_7, connection)


,Название региона,vacancies_count
0,Москва,54.0
1,Санкт-Петербург,42.0
2,Екатеринбург,39.0
3,Нижний Новгород,36.0
4,Новосибирск,35.0
5,Воронеж,32.0
6,Краснодар,30.0
7,Самара,26.0
8,Уфа,26.0
9,Казань,25.0


***

По результатам анализа работодателей можно подтвердить гипотезу, что в базе присутствуют работодатели из нескольких стран (Россия, Казахстан, Беларусь и даже Австралия). При этом некоторые регионы показывают высокую долю регистрации работодателей при нулевом количестве вакансий, в основном это относится к регионам обобщенным (страна, область), что говорит о потребности более точно указывать регион вакансии. Это подтверждается количеством вакансий с типом занятости Полная занятость из предыдущего раздела. Очевидно, что крупные города и столицы имеют большее количество вакансий, чем отдаленные и малонаселенные регионы и города. Самой обширной географией вакансий обладает компания Яндекс, также много регионов размещения вакансии имеют банки (сеть филиалов), Почта России (сеть почтовых отделений) и крупные ретейлеры. В базе присутствуют работодатели, которые не указали сферу деятельности - объяснить с чем это связано нет возможности (номенклатура компаний очень разная и как-то их объединить - довольно сложно). Возможно это связано с обновлением платформы после размещения вакансии, возможно с особенностью hr отделов данных компания либо их отсутствием. Также присутствуют компании с большей чем 1 сферой деятельности, что характерно для крупных компаний. Для компаний поменьше необходимо дополнительное исследование, чтобы определить зависимость количества сфер детальности от размера (региона, страны и т.д) компании.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [40]:
query_6_1 = f'''
            select
                count(v.id)
            from
                public.vacancies as v
            where v.name ilike '%data%' or v.name ilike '%данн%'
'''


In [41]:
data_vaccancies = pd.read_sql_query(query_6_1, connection)
data_vaccancies

C:\Users\22775\AppData\Local\Temp\ipykernel_51304\2431649644.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_vaccancies = pd.read_sql_query(query_6_1, connection)


,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [42]:
query_6_2 = f'''
            select
                count(v.id)
            from
                public.vacancies as v
            where
                (v.name ilike any ('
                {{%data scientist%, %data science%, %исследователь данных%, 
                %machine learning%, %machine learning%, %машинн%обучен%}}
                ') or 
                (name not like '%HTML%' and name like '%ML%'))
                and
                (v.name ilike '%junior%'
                or v.experience = 'Нет опыта'
                or v.employment = 'Стажировка')
'''


In [43]:
junior_ds_vaccancies = pd.read_sql_query(query_6_2, connection)
junior_ds_vaccancies

C:\Users\22775\AppData\Local\Temp\ipykernel_51304\3823487580.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  junior_ds_vaccancies = pd.read_sql_query(query_6_2, connection)


,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [44]:
query_6_3 = f'''
            select
                count(v.id)
            from
                public.vacancies as v
            where
                (v.name ilike any ('
                {{%data scientist%, %data science%, %исследователь данных%, 
                %machine learning%, %machine learning%, %машинн%обучен%}}
                ') or 
                (name not like '%HTML%' and name like '%ML%'))
                and
                (v.key_skills ilike '%postgres%'
                or v.key_skills ilike '%SQL%')
'''

In [45]:
sql_vaccancies = pd.read_sql_query(query_6_3, connection)
sql_vaccancies

C:\Users\22775\AppData\Local\Temp\ipykernel_51304\2090608564.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sql_vaccancies = pd.read_sql_query(query_6_3, connection)


,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [46]:
query_6_4 = f'''
            select
                count(v.id)
            from
                public.vacancies as v
            where
                (v.name ilike any ('
                {{%data scientist%, %data science%, %исследователь данных%, 
                %machine learning%, %machine learning%, %машинн%обучен%}}
                ') or 
                (name not like '%HTML%' and name like '%ML%'))
                and
                v.key_skills ilike '%Python%'
'''

In [47]:
python_vaccancies = pd.read_sql_query(query_6_4, connection)
python_vaccancies

C:\Users\22775\AppData\Local\Temp\ipykernel_51304\2822598272.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  python_vaccancies = pd.read_sql_query(query_6_4, connection)


,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [48]:
query_6_5 = f'''
        select
            round(avg(length(key_skills) -length(replace(key_skills, CHR(9), ''))+1), 2) as "Количество ключевых навыков (сред)"
        from
            public.vacancies as v
        where
            (v.name ilike any ('
            {{%data scientist%, %data science%, %исследователь данных%, 
            %machine learning%, %machine learning%, %машинн%обучен%}}
            ') or 
            (name not like '%HTML%' and name like '%ML%'))
'''

In [49]:
key_skills = pd.read_sql_query(query_6_5, connection)
key_skills

C:\Users\22775\AppData\Local\Temp\ipykernel_51304\1265444755.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  key_skills = pd.read_sql_query(query_6_5, connection)


,Количество ключевых навыков (сред)
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [50]:
query_6_6 = f'''
            select
                v.experience,
                round(avg(coalesce((v.salary_from+v.salary_to)/2, v.salary_from, v.salary_to)), 0)
            from
                public.vacancies as v
            where
                (v.name ilike any ('
                {{%data scientist%, %data science%, %исследователь данных%, 
                %machine learning%, %machine learning%, %машинн%обучен%}}
                ') or 
                (name not like '%HTML%' and name like '%ML%'))
            group by v.experience
'''

In [51]:
avg_sallary_ds = pd.read_sql_query(query_6_6, connection)
avg_sallary_ds

C:\Users\22775\AppData\Local\Temp\ipykernel_51304\4185382056.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  avg_sallary_ds = pd.read_sql_query(query_6_6, connection)


,experience,round
0,Более 6 лет,NaN
1,Нет опыта,74643.0
2,От 1 года до 3 лет,139675.0
3,От 3 до 6 лет,243115.0


***

По результатам предметного анализа мы видим, что из 49197 вакансий 1771 вакансия имеет отношение к данным, что составляет приблизительно 3,6% от общего количества. 51 вакансия для специалистов в области DS с отсутствием опыта (уровень junior), что совсем немного в рамках рассматриваемой базы. Такие вакансии, в случае указания размера оплаты труда, предлагают в среднем 74 000 рублей. Вакансии с требованием большего опыта предлагают больше размер оплаты труда: 140 000 рублей с опытом от 1 до 3 лет и 243 000 рублей с опытом от 3 до 6 лет, что показывает рост с практически геометрической прогрессией. Работодатели хотят видеть в специалистах DS владение не только Python (351 вакансия), но и SQL либо Postgres (201 вакансия), а в среднем количество разных ключевых навыков, требуемых работодателем, составляет 6,41 на вакансию. Это позволяет сделать очевидный вывод о том, что специалист, обладающий большим количеством навыков, имеет больший выбор вакансий. Что обусловлено спецификой работы в области DS - необходимо не только работать с языками програмирования, но и разбираться в предметной области и обладать навыками получения данных из разных источников.

# Общий вывод по проекту

Общий вывод сделаем довольно лаконичным. База Имеет более 49 тысяч вакансий, практически все вакансии относятся к IT сфере. Самая распространенная вакансия - Системный администратор (1,761) на втором месте Программист 1С (1,153). Скорее всего это обобщенные названия, которые будут включать в себя множество обязанностей. В столицах топ 3 вакансий это различные аналитики (Системный аналитик, Бизнес-аналитик, Аналитик). В городах поменьше на первые места выходят Системные администраторы. По моему мнению эти данные показывают, что уровенб развития IT в столицах немного выше и поэтому там ищут специалистов узкого профиля, в отличие от городов поменьше, где нужны многопрофильные специалисты. Хотя если посмотреть в разрезе крупных компаний с вакансиями в регионах - там будут так же присутствовать вакансии узкой специализации.
Сложно давать прогнозы, здесь нет временных рядов и привязать что-то к тенденции не вижу возможности. Могу лишь отметить, что присутствуют много вакансий с разным названием, но по сути являющимися одинаковыми.

Дополнительно провёл исследование зависимости средней указанной заработной платы от региона и количество вакансий в регионе. Самая высокая указана в Германии 452,751 рублей (по курсу скорее всего). При этом самое там же и самое большое количество вакансий в ЕС. А больше всего в базе вакансий загранице в Грузии (Тбилиси). По РФ ожидаемо в Москве самая высока заработная плата и количество вакансий, далее идут крупные города и областные центры с уровнем зп ниже примерно на 30%. Связано, видимо, с уровнем жизни в регионах.
Также мне стало интересно как по регионам распределены графики работы, и я обнаружил зависимость, что соотношение Полной занятости и вакансий удаленной работы приблизительно 1 к 4. Это говорит, что довольно большая доля вакансий подразумевает удаленную работы и скорее всего количество таких вакансий будет расти.

Мне бы хотелось видеть дату размещения вакансии, и маркер обновлялась ли она. Чтобы видеть тенденции по каждой вакансии (изменяется ли зп, график). Постоянно ли требует в конкретную крупную компанию такие специалисты. Чтобы понимать есть ли текучка кадров и колебания заработной платы либо тенденции к её увеличению или уменьшению. Чтобы количество вакансий можно было привязать количественно ко времени и видеть изменение количества во времени.

В целом база дала понять, что рынок труда в IT сфере довольно многогранен и что присутствует возможность для IT специалиста работать удаленно. И по уровню заработной платы в среднем будет превосходить обычные офисные и линейные специальности